In [25]:
import numpy as np
import matplotlib as mpl
import cmocean
import netCDF4
import matplotlib.pyplot as plt
import os
from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import bisect

In [26]:
#User changes
model = 'pong'
source = 'pong.tamu.edu'

#PATHS
#Grid
grid_path = 'http://barataria.tamu.edu:8080/thredds/dodsC/txla_nesting6_grid/txla_grd_v4_new.nc'
#oxygen
pong_path = '/Users/vrx/copano/output_10yr_mpdata/' #local
#pong_path = 'http://barataria.tamu.edu:8080/thredds/dodsC/oofv2_his/out/files/'
sura_path='http://comt.sura.org/thredds/dodsC/comt_2_full/gom_hypoxia/DAL_ROMS/simpleO2model/' \
          'run4_SOC_with_fixed_temperature_large_grid/Output/his_mch_txla007e2_0'
#River discharge
river_path = '/Users/vrx/copano/inputs/rivers/txla2_river_'
#Wind
wind_path = '/Users/vrx/copano/inputs/'
#Save to
save_path= './'


In [27]:
#Plot stuff

# Colormap for model output
cmap = cmocean.cm.oxygen
cmin = 0; cmax = 300; dc = 60
ticks = np.arange(cmin, cmax+dc, dc)

cdx = 7; cdy = 11 # in indices

# mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 14})
mpl.rcParams['font.sans-serif'] = 'Arev Sans, Bitstream Vera Sans, Lucida Grande, Verdana, Geneva, Lucid, Helvetica, Avant Garde, sans-serif'
mpl.rcParams['mathtext.fontset'] = 'custom'
mpl.rcParams['mathtext.cal'] = 'cursive'
mpl.rcParams['mathtext.rm'] = 'sans'
mpl.rcParams['mathtext.tt'] = 'monospace'
mpl.rcParams['mathtext.it'] = 'sans:italic'
mpl.rcParams['mathtext.bf'] = 'sans:bold'
mpl.rcParams['mathtext.sf'] = 'sans'
mpl.rcParams['mathtext.fallback_to_cm'] = 'True'

In [4]:
def background(lon_r,lat_r,h,basemap, ax=None, pars=np.arange(18, 35), mers=np.arange(-100, -80), 
                hlevs=np.hstack(([10,20],np.arange(50,500,50))), 
                col='lightgrey', fig=None, outline=[1, 1, 0, 1], merslabels=[0, 0, 0, 1],
                parslabels=[1, 0, 0, 0]):
    """
    Plot basic TXLA shelf background: coastline, bathymetry, meridians, etc
    Can optionally input grid (so it doesn't have to be loaded again)

    pars    parallels to plot
    mers    meridians to plot
    hlevs   which depth contours to plot
    outline     west, east, north, south lines (left, right, top, bottom)
    """
    
    xr,yr = basemap(lon_r,lat_r)

    if fig is None:
        fig = gcf()

    if ax is None:
        ax = gca()

    # Do plot   
    basemap.drawcoastlines(ax=ax)
    basemap.fillcontinents(color = 'slategrey',ax=ax)
    basemap.drawparallels(pars, dashes=(1, 1), 
                            linewidth=0.15, labels=parslabels, ax=ax)
    basemap.drawmeridians(mers, dashes=(1, 1), 
                            linewidth=0.15, labels=merslabels, ax=ax)
    # hold('on')
    ax.contour(xr,yr,h, hlevs, colors=col, linewidths=0.5)

    # Outline numerical domain
    # if outline:  # backward compatibility
    #     outline = [1,1,1,1]
    if outline[3]:
        ax.plot(xr[0,:], yr[0,:], 'k:')
    if outline[2]:
        ax.plot(xr[-1,:], yr[-1,:], 'k:')
    if outline[1]:
        ax.plot(xr[:,-1], yr[:,-1], 'k:')
    if outline[0]:
        ax.plot(xr[:,0], yr[:,0],'k:')

In [23]:
###################################################################################################
def get_btmox(model, year=None, ini=1, end=None):

    """
    Fetch bottom oxygen data from pong or sura servers
    get_btmox(model, year=None, ini=1, end=None)

    Input:
    model : 'pong' or 'sura', strings
    year : only and required for pong
    ini : the initial number of the threads server catalog. Default is 1.
    end : the last numer of the threads server catalog. Defaults are set as maximum for pong and sura.
    ini and end can be modified to get only specific dates

    Output:
    btm_ox : bottom oxygen array with dimensions time, x_rho, y_rho
    modeltimes: ocean time
    model_dates : datetime array
    """

    try: units
    except NameError:
        pass
    else:
        del units
    if model=='pong':
        if year==None:
            print "for pong model need to enter year"
        else:
            if end==None: end=26
            path = pong_path + str(year)+'/ocean_his_00'
            catalog = ["%02d" % i for i in range(ini,end)]
    elif model=='sura':
        path = sura_path
        if n==None: n=220
        catalog = ["%03d" % i for i in range(ini,end)]        
    else:
        print 'only pong and sura models available'
        quit()

    print 'fetching data from %s server' % model
    for item in catalog:
        total_path = path+item+'.nc'
        print total_path
        nc = netCDF4.Dataset(total_path)
        try: units
        except NameError:
            print 'initializing variables'
            units = nc.variables['ocean_time'].units
#             model_times = nc.variables['ocean_time'][:]
            btm_ox=np.empty((nc.variables['dye_01'][:,0,0,0][0],np.shape(nc.variables['dye_01'][0,0,:,:])[0],np.shape(nc.variables['dye_01'][0,0,:,:])[1]))
            for t in range(np.shape(nc.variables['dye_01'][:,0,0,0])[0]):
                print t
                btm_ox = nc.variables['dye_01'][t,0,:,:]
#                 make_plot(plotdates(len(btm_ox[:,0,0])))
#             btm_ox = nc.variables['dye_01'][:,0,:,:]
#             model_times = nc.variables['ocean_time'][:]
        else:
#             model_times_add = nc.variables['ocean_time'][:]
#             btm_ox_add=np.empty((np.size(model_times),np.shape(nc.variables['dye_01'][0,0,:,:])[0],np.shape(nc.variables['dye_01'][0,0,:,:])[1]))
            for t in range(np.shape(nc.variables['dye_01'][:,0,0,0])[0]):
                btm_ox = np.vstack((btm_ox, nc.variables['dye_01'][t,0,:,:]))
#                 make_plot(plotdates(len(btm_ox[:,0,0])))
#                 btm_ox_add = nc.variables['dye_01'][t,0,:,:]
#             btm_ox = np.vstack((btm_ox, btm_ox_add))
#             model_times = np.hstack((model_times,model_times_add) )

    print 'extracted from %d files' % (int(catalog[-1])-int(catalog[0])+1)
    nc.close()
#     model_dates = netCDF4.num2date(model_times, units)
    np.save('btm_ox', btm_ox)
#     np.save('model_times', model_times)
#     np.save('model_dates', model_dates )

    return btm_ox 
#     return btm_ox, model_times, model_dates, units

In [6]:
###################################################################################################
def get_alldates(model, year=None, ini=1, end=None):

    """
    Fetch bottom oxygen data from pong or sura servers
    get_btmox(model, year=None, ini=1, end=None)

    Input:
    model : 'pong' or 'sura', strings
    year : only and required for pong
    ini : the initial number of the threads server catalog. Default is 1.
    end : the last numer of the threads server catalog. Defaults are set as maximum for pong and sura.
    ini and end can be modified to get only specific dates

    Output:
    btm_ox : bottom oxygen array with dimensions time, x_rho, y_rho
    modeltimes: ocean time
    model_dates : datetime array
    """

    try: units
    except NameError:
        pass
    else:
        del units
    if model=='pong':
        if year==None:
            print "for pong model need to enter year"
        else:
            if end==None: end=26
            path = pong_path + str(year)+'/ocean_his_00'
            catalog = ["%02d" % i for i in range(ini,end)]
    elif model=='sura':
        path = sura_path
        if n==None: n=220
        catalog = ["%03d" % i for i in range(ini,end)]        
    else:
        print 'only pong and sura models available'
        quit()

    print 'fetching data from %s server' % model
    for item in catalog:
        total_path = path+item+'.nc'
        print total_path
        nc = netCDF4.Dataset(total_path)
        m=1
        try: units
        except NameError:
            print 'initializing variables'
            units = nc.variables['ocean_time'].units
            model_times = nc.variables['ocean_time'][:]
        else:
            model_times = np.hstack((model_times,nc.variables['ocean_time'][:]) )
    print 'extracted from %d files' % (int(catalog[-1])-int(catalog[0])+1)
    nc.close()
    model_dates = netCDF4.num2date(model_times, units)

    np.save('model_times', model_times)
    np.save('model_dates', model_dates )
    
    starttime = netCDF4.date2num(datetime(year, 1, 1, 4, 0, 0), units)
    if year==2016:
        endtime = netCDF4.date2num(datetime(year, 10, 30, 20, 0, 0), units)
    else:
        endtime = netCDF4.date2num(datetime(year+1, 1, 1, 4, 0, 0), units)
    dt = model_times[1] - model_times[0] # 4 hours in seconds
    ts = np.arange(starttime, endtime, dt)
    
    return model_times, model_dates, units, netCDF4.num2date(ts, units)

In [7]:
# Grid info

grd_path = grid_path
grd = netCDF4.Dataset(grd_path)

h = grd.variables['h'][:]

xpsi = grd.variables['x_psi'][:]
ypsi = grd.variables['y_psi'][:]

lon_rho = grd.variables['lon_rho'][:]
lat_rho = grd.variables['lat_rho'][:]

llcrnrlat=22.85
llcrnrlon=-97.9
urcrnrlat=30.5
urcrnrlon=-87.5
lat_0 = (llcrnrlat+urcrnrlat)*0.5
lon_0 = (llcrnrlon+urcrnrlon)*0.5
res='i'
projection='lcc'

my_map = Basemap(llcrnrlon=llcrnrlon,
                         llcrnrlat=llcrnrlat,
                         urcrnrlon=urcrnrlon,
                         urcrnrlat=urcrnrlat,
                         projection=projection,
                         lat_0=lat_0,
                         lon_0=lon_0,
                         resolution=res)

x_map,y_map = my_map(lon_rho,lat_rho)

In [8]:
def get_river_data(year):
    '''
    Get river discharge data and times for a specific year
    Input: year
    Ouput: dishcarge (1D array in time), times, and dates (datetime array)
    [dates run from Dic 31 previous year, up Jan 01 next year]
    '''
    
        
    rPATH = river_path+str(year)+'_AR_newT_SWpass_weekly.nc'
    rnc = netCDF4.Dataset(rPATH)

    r_times = rnc['river_time'][:]
    r_dates = netCDF4.num2date(r_times, rnc['river_time'].units)
    transport = rnc['river_transport']
    Q = np.abs(transport[:,0:43]).sum(axis=1)
    rnc.close()
    
    np.save('Q', Q)
    np.save('r_times', r_times)
    np.save('r_dates', r_dates)
    return Q, r_times, r_dates

In [9]:
def get_wspeed_data(year):
    '''
    Get wind speed data and times for a specific year
    Input: year
    Ouput: wind speed (1D, spatial average in time), times, and dates (datetime array)
    [dates run from Dic 31 previous year, up Jan 01 next year]
    '''
    
    wPATH = wind_path+str(year)+'/txla_bulk_ERAI_'+str(year)+'.nc'
    wnc = netCDF4.Dataset(wPATH)

    w_times = wnc['time'][:]
    w_dates = netCDF4.num2date(w_times, wnc['time'].units)
    wspeed = wnc['wspd']
    wspd = wspeed[:].mean(axis=(1,2))
    wnc.close()
    
    np.save('wspd', wspd)
    np.save('w_times', w_times)
    np.save('w_dates', w_dates)
    return wspd, w_times, w_dates

In [10]:
def get_plotdates(year, units):
    """Get Model time period to use for the plots"""    

    starttime = netCDF4.date2num(datetime(year, 1, 1, 4, 0, 0), units)
    if year==2016:
        endtime = netCDF4.date2num(datetime(year, 10, 30, 20, 0, 0), units)
    else:
        endtime = netCDF4.date2num(datetime(year+1, 1, 1, 4, 0, 0), units)
    dt = model_times[1] - model_times[0] # 4 hours in seconds
    ts = np.arange(starttime, endtime, dt)
  
    return netCDF4.num2date(ts, units)

In [11]:
def make_plot(plotdate):
    """docstring for make_plot"""

    # Set up before plotting
    itmodel = bisect.bisect_left(model_dates, plotdate) # index for model output at this time
    itwind = bisect.bisect_left(w_dates, plotdate) # index for wind at this time
    itriver = bisect.bisect_left(r_dates, plotdate) # index for river at this time

    figname = save_path+str(year)+'/'+ model_dates[itmodel].isoformat()[0:13]+'.png'

#     # Don't redo plot
#     if os.path.exists(figname):
#         continue
    
    # Set up plot
    fig = plt.figure(figsize=(11, 9), dpi=100) # Fig site
    ax = fig.add_axes([0.06, 0.03, 0.92, 0.97]) # Background size
    ax.set_frame_on(False) # Remove box

    # plot base map
    background(lon_rho,lat_rho,h,my_map, fig=fig, ax=ax, mers=np.arange(-97, -87), pars=np.arange(23, 32)) 

    # Label isobaths
    ax.text(0.85, 0.865, '10 m', transform=ax.transAxes, fontsize=9, color='0.4', rotation=45)
    ax.text(0.88, 0.862, '20'  , transform=ax.transAxes, fontsize=9, color='0.4', rotation=45)
    ax.text(0.87, 0.835, '50'  , transform=ax.transAxes, fontsize=9, color='0.4', rotation=45)
    ax.text(0.89, 0.825, '100' , transform=ax.transAxes, fontsize=9, color='0.4', rotation=45)
    ax.text(0.9 , 0.803, '450' , transform=ax.transAxes, fontsize=9, color='0.4', rotation=45)

    # Date label
    d_lb = model_dates[itmodel].strftime('%Y - %b - %d %H:%M')
    ax.text(0.4, 0.1, d_lb, fontsize=16, color='0.2', transform=ax.transAxes, 
              bbox=dict(facecolor='white', edgecolor='white', boxstyle='round'))

    # source
    ax.text(0.45, 0.97, source, fontsize=12, transform=ax.transAxes, color='0.2')

    #Plot bottom ox
    mappable = ax.pcolormesh(x_map, y_map, btm_ox[itmodel,:,:], cmap=cmap, vmin=cmin, vmax=cmax)

    # Colorbar in upper left corner
    cax = fig.add_axes([0.07, 0.93, 0.35, 0.025]) #colorbar axes
    cb = fig.colorbar(mappable, cax=cax, orientation='horizontal')
    cb.set_label(r'Bottom oxygen [g$\cdot$kg$^{-1}$]', fontsize=11, color='0.2')
    cb.ax.tick_params(labelsize=11, length=2, color='0.2', labelcolor='0.2') 
    cb.set_ticks(ticks)

    # Plot Mississippi river discharge rate
    #Frame
    axr = fig.add_axes([0.4, 0.20, 0.55, .15])       
    for axis in ['top','left','right']:
        axr.spines[axis].set_linewidth(0.05)
    axr.spines['bottom'].set_linewidth(0.0)    

    #fill with every time step
    # make background rectangle so lines don't overlap
    axr.fill_between(r_times[1:itriver+1], Q[1:itriver+1], alpha=0.5, facecolor='0.4', edgecolor='0.4', zorder=2)
    axr.plot(r_times[1:itriver], Q[1:itriver], '-', color='0.4')
    axr.plot(r_times[1:], Q[1:], '-', color='0.4', alpha=0.3)
    axr.plot([r_times[1], r_times[-1]], [5, 5], '-', color='0.6', lw=0.5, alpha=0.5)
    axr.plot([r_times[1], r_times[-1]], [10000, 10000], '-', color='0.6', lw=0.5, alpha=0.5)
    axr.plot([r_times[1], r_times[-1]], [20000, 20000], '-', color='0.6', lw=0.5, alpha=0.5)
    axr.plot([r_times[1], r_times[-1]], [30000, 30000], '-', color='0.6', lw=0.5, alpha=0.5)

    # this makes sure alignment stays consistent in different years
    axr.autoscale(axis='x', tight=True) 
    axr.set_ylim(-1000,45000) 

    #Make ticks for the river plot
    if year == 2016:
        monthdates = [datetime(year, month, 1, 0, 0, 0) for month in np.arange(1,10)]
    else:
        monthdates = [datetime(year, month, 1, 0, 0, 0) for month in np.arange(1,13)]
    mticks = [bisect.bisect_left(r_dates, monthdate) for monthdate in np.asarray(monthdates)]
    mticknames = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']

    # labels
    axr.text(r_times[mticks[-3]]+16.5, 5, '0', fontsize=9, color='0.4', alpha=0.7)
    axr.text(r_times[mticks[-3]]+16.5, 10000, '10', fontsize=9, color='0.4', alpha=0.7)
    axr.text(r_times[mticks[-3]]+16.5, 20000, '20', fontsize=9, color='0.4', alpha=0.7)
    axr.text(r_times[mticks[-3]]+15, 30000, r'$30\times10^3$ m$^3$s$^{-1}$', fontsize=9, color='0.4', alpha=0.7)
    axr.text(r_times[mticks[-7]]+15, 30000, 'Mississippi discharge', fontsize=10, color='0.2')

    #ticks
    axr.get_xaxis().set_ticklabels([])
    axr.xaxis.set_ticks_position('bottom')

    #add ticks for each month
    axr.set_xticks(r_times[mticks])
    axr.tick_params('x', width=1.5, color='0.4') # make ticks bigger
    axr.get_yaxis().set_visible(False)
    axr.get_xaxis().set_visible(False)

    #label month ticks
    for i in xrange(len(mticks)):
        #axr.text(tRiver[mticks[i]], 2500, mticknames[i], fontsize=9, color='0.2')
        axr.text(r_times[mticks[i]], 2500, mticknames[i], fontsize=9, color='0.2')

    axr.add_patch( mpl.patches.Rectangle( (0.3, 0.162), 0.7, 0.2, transform=ax.transAxes, color='white', zorder=1))    

    # Wind speed time series
    #Frame
    axw = fig.add_axes([0.4, 0.35, 0.55, .15])       
    for axis in ['top','left','right']:
        axw.spines[axis].set_linewidth(0.05)
    axw.spines['bottom'].set_linewidth(0.0)    

    axw.fill_between(w_times[1:itwind+1], wspd[1:itwind+1], alpha=0.5, facecolor='0.4', edgecolor='0.4', zorder=2)
    axw.plot(w_times[0:itwind], wspd[0:itwind], '-', color='k')
    axw.plot(w_times[0:], wspd[0:], '-', color='0.4', alpha=0.3)
    axw.plot([w_times[0], w_times[-1]], [5, 5], '-', color='0.6', lw=0.5, alpha=0.5)
    axw.plot([w_times[0], w_times[-1]], [10, 10], '-', color='0.6', lw=0.5, alpha=0.5)
    axw.plot([w_times[0], w_times[-1]], [15, 15], '-', color='0.6', lw=0.5, alpha=0.5)
    axw.plot([w_times[0], w_times[-1]], [20, 20], '-', color='0.6', lw=0.5, alpha=0.5)

    # this makes sure alignment stays consistent in different years
    axw.autoscale(axis='x', tight=True) 
    axw.set_ylim(-1,25)

    mticks = [bisect.bisect_left(w_dates, monthdate) for monthdate in np.asarray(monthdates)]

    # labels
    axw.text(w_times[mticks[-3]]+16.5, 5, '5', fontsize=9, color='0.4', alpha=0.7)
    axw.text(w_times[mticks[-3]]+16.5, 10, '10', fontsize=9, color='0.4', alpha=0.7)
    axw.text(w_times[mticks[-3]]+16.5, 15, '15', fontsize=9, color='0.4', alpha=0.7)
    axw.text(w_times[mticks[-3]]+15, 20, r'$20 m$s$^{-1}$', fontsize=9, color='0.4', alpha=0.7)
    axw.text(w_times[mticks[-7]]+15, 20, 'Wind speed', fontsize=10, color='0.2')

    #ticks
    axw.get_xaxis().set_ticklabels([])
    axw.xaxis.set_ticks_position('bottom')

    #add ticks for each month
    axw.set_xticks(w_times[mticks])
    axw.tick_params('x', width=1.5, color='0.4') # make ticks bigger
    axw.get_yaxis().set_visible(False)
    axw.get_xaxis().set_visible(False)

    #label month ticks
    for i in xrange(len(mticks)):
        axw.text(w_times[mticks[i]], 2500, mticknames[i], fontsize=9, color='0.2')

    axw.add_patch( mpl.patches.Rectangle( (0.3, 0.162), 0.7, 0.2, transform=ax.transAxes, color='white', zorder=1))    

    #save figure
    plt.savefig(figname)
    plt.close(fig)

In [12]:
#Fetch data
year = 2015

print "River discharge"
if os.path.exists(save_path+'/Q.npy'):
    Q = np.load(save_path+'/Q.npy')
    r_times = np.load(save_path+'/r_times.npy')
    r_dates = np.load(save_path+'/r_dates.npy')
    print "local"
else:
    Q, r_times, r_dates = get_river_data(year)
    
# print "River discharge"
# if os.path.exists(save_path+str(year)+'/Q.npy'):
#     Q = np.load(save_path+str(year)+'/Q.npy')
#     r_times = np.load(save_path+str(year)+'/r_times.npy')
#     r_dates = np.load(save_path+str(year)+'/r_dates.npy')
#     print "local"
# else:
#     Q, r_times, r_dates = get_river_data(year)
    

River discharge
local


In [13]:
print "Wind speed"
if os.path.exists(save_path+'/wspd.npy'):
    wspd = np.load(save_path+'/wspd.npy')
    w_times = np.load(save_path+'/w_times.npy')
    w_dates = np.load(save_path+'/w_dates.npy')    
    print "local"
else:
    wspd, w_times, w_dates = get_wspeed_data(year)
    
# print "Wind speed"
# if os.path.exists(save_path+str(year)+'/wspd.npy'):
#     wspd = np.load(save_path+str(year)+'/wspd.npy')
#     w_times = np.load(save_path+str(year)+'/w_times.npy')
#     w_dates = np.load(save_path+str(year)+'/w_dates.npy')    
#     print "local"
# else:
#     wspd, w_times, w_dates = get_wspeed_data(year)

Wind speed
local


In [17]:
if os.path.exists(save_path+'/model_times.npy'):
    model_times = np.load(save_path+'/model_times.npy')
    model_dates = np.load(save_path+'/model_dates.npy')
    units = netCDF4.Dataset(pong_path + str(year)+'/ocean_his_0001.nc').variables['ocean_time'].units
    plotdates = get_plotdates(year, units)
    print "local"
else:
    model_times, model_dates, units, plotdates = get_alldates(model, year)
# if os.path.exists(save_path+str(year)+'/model_times.npy'):
#     model_times = np.load(save_path+str(year)+'/model_times.npy')
#     model_dates = np.load(save_path+str(year)+'/model_dates.npy')
#     units = netCDF4.Dataset(pong_path + str(year)+'/ocean_his_0001').variables['ocean_time'].units
#     plotdates = get_plotdates(year, units)
#     print "local"
# else:
#     model_times, model_dates, units, plotdates = get_alldates(model, year)

local


In [24]:
# print "Time period"
# plotdates = get_plotdates(year, units)

print "Bottom oxygen concentrations"
if os.path.exists(save_path+'/btm_ox.npy'):
    btm_ox = np.load(save_path+'/btm_ox.npy')
    print "local"
# if os.path.exists(save_path+str(year)+'/btm_ox.npy'):
#     btm_ox = np.load(save_path+str(year)+'/btm_ox.npy')
#     model_times = np.load(save_path+str(year)+'/model_times.npy')
#     model_dates = np.load(save_path+str(year)+'/model_dates.npy')
#     units = netCDF4.Dataset(pong_path + str(year)+'/ocean_his_0001.nc').variables['ocean_time'].units
#     print "local"
else:
#     btm_ox, model_times, model_dates, units = get_btmox(model,year)
    btm_ox = get_btmox(model,year)
# #Make plots
# print "Making plots!"
# for plotdate in plotdates:
#     print "Ploting", plotdate
#     make_plot(plotdate)

print "DONE"

print "to generate the movie move to ", source
print "then execute"
print "ffmpeg -y -r 20 -pattern_type glob -i '2006*.png' -vcodec libx264 -pix_fmt yuv420p  -crf 15 movie_2006v4.mp4"


Bottom oxygen concentrations
fetching data from pong server
/Users/vrx/copano/output_10yr_mpdata/2015/ocean_his_0001.nc
initializing variables
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241

/Users/vrx/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:51: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


RuntimeError: NetCDF: HDF error

In [5]:
# def resize(A, dim):
#     """
#     Average neighboring elements in an array A along a dimension dim.
#     Args:
#         A (array): array size, [m x n] in 2D case. Can be up to 3D.
#         dim (int): dimension on which to act. Can be up to 2 (0, 1, 2).
#     Returns:
#         * A - array of size [(m-1) x n] if dim is 0
#     """

#     # B is A but rolled such that the dimension that is to be resized is in
#     # the 0 position
#     B = np.rollaxis(A, dim)

#     # Do averaging
#     B = 0.5*(B[0:-1]+B[1:])

#     # Roll back to original
#     return np.rollaxis(B, 0, dim+1)

In [6]:
# def get_river_indx(river_dates, year):
#     '''
#     Get start and end indices in time for river discharge for a specific year
#     Input: datetime array
#     Ouput: itstartRiver, itendRiver
#     '''
#     itstartRiver = bisect.bisect_left(datesRiver, datetime(year, 1, 1, 0, 0, 0))
#     if year == 2014:
#         itendRiver = bisect.bisect_left(datesRiver, datetime(year, 9, 30, 20, 0, 0))
#     else:
#         itendRiver = bisect.bisect_left(datesRiver, datetime(year+1, 1, 1, 0, 0, 0))
    
#     return itstartRiver, itendRiver


In [42]:
# ## River forcing  data##
# r1 = netCDF4.Dataset('/Users/vrx/Projects/riverdata/TXLA_river_4dyes_2012.nc') # use for through 2011
# r2 = netCDF4.Dataset('/Users/vrx/Projects/riverdata/TXLA_river_4dyes_2012_2014.nc') # use for 2012-2014
# # River timing
# r1_times = r1.variables['river_time'][:] 
# r1_times = resize(r1_times, 0)
# r1_dates = netCDF4.num2date(r1_times, r1.variables['river_time'].units)
# r2_times = r2.variables['river_time'][:] 
# r2_dates = netCDF4.num2date(r2_times, r1.variables['river_time'].units)

# # all of river input
# Q1 = np.abs(r1.variables['river_transport'][:]).sum(axis=1)*2.0/3.0
# Q1 = resize(Q1, 0)
# Q2 = np.abs(r2.variables['river_transport'][:]).sum(axis=1)*2.0/3.0
# # Combine river info into one dataset
# iend1 = np.where(r1_dates<datetime(2012,1,1,0,0,0))[0][-1] # ending index for file 1
# tRiver = np.concatenate((r1_times[:iend1], r2_times), axis=0)
# datesRiver = np.concatenate((r1_dates[:iend1], r2_dates))
# R = np.concatenate((Q1[:iend1], Q2))
# r1.close(); r2.close()

#     # ticks for months on river discharge
# # mticks = [bisect.bisect_left(datesRiver, monthdate) for monthdate in np.asarray(monthdates)]
# # mticknames = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']
# ##


In [44]:
#itstartRiver, itendRiver = get_river_indx(datesRiver, 2013)

In [ ]:
###################################################################################################
def get_btmox(model, year=None, ini=1, end=None):

    """
    Fetch bottom oxygen data from pong or sura servers
    get_btmox(model, year=None, ini=1, end=None)

    Input:
    model : 'pong' or 'sura', strings
    year : only and required for pong
    ini : the initial number of the threads server catalog. Default is 1.
    end : the last numer of the threads server catalog. Defaults are set as maximum for pong and sura.
    ini and end can be modified to get only specific dates

    Output:
    btm_ox : bottom oxygen array with dimensions time, x_rho, y_rho
    modeltimes: ocean time
    model_dates : datetime array
    """

    try: units
    except NameError:
        pass
    else:
        del units
    if model=='pong':
        if year==None:
            print "for pong model need to enter year"
        else:
            if end==None: end=26
            path = pong_path + str(year)+'/ocean_his_00'
            catalog = ["%02d" % i for i in range(ini,end)]
    elif model=='sura':
        path = sura_path
        if n==None: n=220
        catalog = ["%03d" % i for i in range(ini,end)]        
    else:
        print 'only pong and sura models available'
        quit()

    print 'fetching data from %s server' % model
    for item in catalog:
        total_path = path+item+'.nc'
        print total_path
        nc = netCDF4.Dataset(total_path)
        m=1
        try: units
        except NameError:
            print 'initializing variables'
            btm_ox=np.empty((np.shape(nc.variables['dye_01'][:,0,0,0]),np.shape(nc.variables['dye_01'][0,0,:,:])[0],np.shape(nc.variables['dye_01'][0,0,:,:])[1]))
            for t in range(len(np.shape(nc.variables['dye_01'][:,0,0,0])):
                print t
                btm_ox = nc.variables['dye_01'][t,0,:,:]
                make_plot(plotdates[m])
                m = m +1
#             btm_ox = nc.variables['dye_01'][:,0,:,:]
#             model_times = nc.variables['ocean_time'][:]
        else:
            btm_ox_add=np.empty((np.shape(nc.variables['dye_01'][:,0,0,0][0]),np.shape(nc.variables['dye_01'][0,0,:,:])[0],np.shape(nc.variables['dye_01'][0,0,:,:])[1]))
            for t in range(len(np.shape(nc.variables['dye_01'][:,0,0,0][0])):
                btm_ox_add = nc.variables['dye_01'][t,0,:,:]
            btm_ox = np.vstack((btm_ox, btm_ox_add))
            make_plot(plotdates[m])

    print 'extracted from %d files' % (int(catalog[-1])-int(catalog[0])+1)
    nc.close()
    model_dates = netCDF4.num2date(model_times, units)
    np.save('btm_ox', btm_ox)
    np.save('model_times', model_times)
    np.save('model_dates', model_dates )
    
    return btm_ox, model_times, model_dates, units